In [ ]:
import gym
import numpy as np
from gym import spaces
from dd_class import Dragodinde
from dd_class import Elevage

In [ ]:
class ElevageEnv(gym.Env):
    def __init__(self, elevage):
        super(ElevageEnv, self).__init__()
        self.elevage = elevage

        # Define the action and observation space
        self.action_space = spaces.Discrete(len(elevage.dragodindes) ** 2)
        self.observation_space = spaces.Box(
            low=0, high=10, shape=(len(elevage.dragodindes), 4), dtype=np.float32)

        self.current_step = 0
        self.max_steps = 1000

    def reset(self):
        self.current_step = 0
        self.elevage = self.create_elevage()
        return self._get_observation()

    def create_elevage(self):
        dragodindes_data = [
            (1, "M", "Rousse", 1),
            (2, "F", "Rousse", 1),
            (3, "M", "Amande", 1),
            (4, "F", "Amande", 1),
            (5, "M", "Dorée", 1),
            (6, "F", "Dorée", 1)
        ]

        list_dd = []
        for id, gender, color, generation in dragodindes_data:
            dragodinde = Dragodinde(id, gender, color, generation)
            list_dd.append(dragodinde)

        return Elevage(list_dd)